## Explore Errors Mitigations

In [1]:
import sys
sys.path.append('../../../ResponsibleAIToolBox-Mitigation/')
from errorsmitigation.dataprocessing import DataSplit, DataTransformer, DataRebalance, DataSample

%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm

import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, scale, MinMaxScaler, PowerTransformer


from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


ModuleNotFoundError: No module named 'errorsmitigation'

### Load dataset

In [3]:
data_dir = '../datasets/hr_promotion'
dataset =  pd.read_csv(data_dir + '/train.csv')

seed = 42
dataset.shape
dataset.head()

(54808, 14)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


### Random Sample

In [4]:
dataset_target = dataset.columns.get_loc('is_promoted')

data_sample =  DataSample(dataset, dataset_target, 0.8, False, False, False, True)
random_sample = data_sample.RandomSample()

random_sample.shape
random_sample.head()

c:\Users\Akshara\Desktop\RAI\ErrorsMitigationAPI\notebooks\../../ErrorsMitigationAPI\errorsmitigation\dataprocessing\_data_random_sample.py:82: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  self.dataset.fillna(self.dataset.mean(), inplace=True)


(43846, 14)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
13314,27126,Technology,region_10,Bachelor's,f,sourcing,1,29,4.000000,3,0,0,78,0
30580,44952,Sales & Marketing,region_28,Bachelor's,m,other,1,39,1.000000,4,0,0,50,0
21093,18840,Sales & Marketing,region_20,Bachelor's,m,sourcing,2,33,5.000000,5,0,0,49,0
15164,70282,Technology,region_22,Master's & above,f,other,1,31,4.000000,3,1,0,83,1
10902,16862,Technology,region_25,Bachelor's,f,other,1,29,3.329256,1,0,0,72,0


### Split Dataset 

In [5]:
# dataset - Panda Data Frame. 
# target – A string representing the name of the label column, or the label column integer index (zero base)
# train_size – The training data split size.  The default is 0.9, which split the dataset to 90% training and 10% testing. 
    # Training and Test split values add up to 1. 
# random_state – Control the randomization of the algorithm. 
    # ‘None’: the random number generator is the RandomState instance used by np.random.  
# categorical_features – A Boolean flag to indicates the presence of categorical features. Default is True.  
# drop_null: If flag is set to True, records with null values are dropped, otherwise they are replaced by the mean.
    # Default is True.
# drop_duplicates: if flag is set to True, duplicate records are dropped. Default is False.
# Stratify: If not None, data is split in a stratified fashion, using this as the class labels. Default is False.

random_sample_target = random_sample.columns.get_loc('is_promoted')
data_split =  DataSplit(random_sample, random_sample_target, 0.9, seed, True, False, False, True)

train_data, test_data = data_split.Split()


print(train_data.shape)
print(test_data.shape) 

train_data.head()

(39461, 65)
(4385, 65)


c:\Users\Akshara\Desktop\RAI\ErrorsMitigationAPI\notebooks\../../ErrorsMitigationAPI\errorsmitigation\dataprocessing\_data_split.py:98: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  self.dataset.fillna(self.dataset.mean(), inplace=True)


,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,department_Analytics,...,education_Below Secondary,education_Master's & above,education_nan,gender_f,gender_m,gender_nan,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing,recruitment_channel_nan
13133,31215,1,32,5.000000,6,0,0,64,0,0,...,0,1,0,1,0,0,0,0,1,0
24062,44079,1,41,2.000000,14,0,0,50,0,0,...,0,0,0,0,1,0,0,0,1,0
22905,64058,1,25,3.329256,1,1,0,53,0,0,...,0,0,0,1,0,0,0,0,1,0
19134,26894,1,30,4.000000,3,1,0,97,1,0,...,0,0,0,1,0,0,1,0,0,0
19340,59838,1,34,4.000000,3,0,0,65,0,0,...,0,0,0,0,1,0,0,0,1,0


### Tansform Dataset

In [7]:
# Data Transformation

# dataset,
# target, 
# transformer_type,
# transform_features = None,
# random_state = None,
# method ='yeo-johnson',
# output_distribution  = 'uniform' 
# transform_features = None


# StandardScaler = 1
# MinMaxScaler = 2
# RobustScaler = 3
# PowerTransformer = 4
# QuantileTransformer = 5
# Normalizer = 6

train_data_label = train_data.columns.get_loc('is_promoted')
# train_data.iloc[:,target_index]

dt_train =  DataTransformer(train_data, train_data_label,DataTransformer.TransformerType.StandardScaler, None, seed)
train_data_t = dt_train.Transform()

    
dt_test =  DataTransformer(test_data, train_data_label,DataTransformer.TransformerType.StandardScaler, None, seed)
test_data_t = dt_test.Transform()

train_data.head()
train_data_t.head()


,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,department_Analytics,...,education_Below Secondary,education_Master's & above,education_nan,gender_f,gender_m,gender_nan,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing,recruitment_channel_nan
13133,31215,1,32,5.000000,6,0,0,64,0,0,...,0,1,0,1,0,0,0,0,1,0
24062,44079,1,41,2.000000,14,0,0,50,0,0,...,0,0,0,0,1,0,0,0,1,0
22905,64058,1,25,3.329256,1,1,0,53,0,0,...,0,0,0,1,0,0,0,0,1,0
19134,26894,1,30,4.000000,3,1,0,97,1,0,...,0,0,0,1,0,0,1,0,0,0
19340,59838,1,34,4.000000,3,0,0,65,0,0,...,0,0,0,0,1,0,0,0,1,0


,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0.0,-0.352976,-0.414397,-0.366732,1.373031,0.026589,-0.740828,-0.153207,0.045492,-0.329278,...,-0.123095,1.636976,-0.213026,1.539153,-1.539153,0.0,-1.121381,-0.146939,1.170726,0.0
1,0.0,0.216895,-0.414397,0.807467,-1.099105,1.894804,-0.740828,-0.153207,-1.000163,-0.329278,...,-0.123095,-0.610883,-0.213026,-0.649708,0.649708,0.0,-1.121381,-0.146939,1.170726,0.0
2,0.0,1.101959,-0.414397,-1.279998,-0.003738,-1.141045,1.349841,-0.153207,-0.776094,-0.329278,...,-0.123095,-0.610883,-0.213026,1.539153,-1.539153,0.0,-1.121381,-0.146939,1.170726,0.0
3,1.0,-0.544395,-0.414397,-0.627665,0.548986,-0.673991,1.349841,-0.153207,2.510250,-0.329278,...,-0.123095,-0.610883,-0.213026,1.539153,-1.539153,0.0,0.891758,-0.146939,-0.854171,0.0
4,0.0,0.915014,-0.414397,-0.105799,0.548986,-0.673991,-0.740828,-0.153207,0.120182,-0.329278,...,-0.123095,-0.610883,-0.213026,-0.649708,0.649708,0.0,-1.121381,-0.146939,1.170726,0.0


### Accuracy Results

In [8]:
def split_label_index(dataset):
    x = dataset.drop([0], axis=1)
    y = dataset[0]
    return x, y

def split_label(dataset):
    x = dataset.drop(['is_promoted'], axis=1)
    y = dataset['is_promoted']
    return x, y
# def split_label(dataset):
#     x = dataset.drop(['is_promoted'], axis=1)
#     y = dataset['is_promoted']
#     return x, y

# splitting the training data
x_train, y_train = split_label_index(train_data_t)

# splitting the test data
x_test, y_test = split_label_index(test_data_t)

# LGBMClassifier Model
clf = LGBMClassifier(n_estimators=50)
model = clf.fit(x_train, y_train)

pred = model.predict(x_test)

def conf_matrix(y,pred):
    ((tn, fp), (fn, tp)) = metrics.confusion_matrix(y, pred)
    ((tnr,fpr),(fnr,tpr))= metrics.confusion_matrix(y, pred, normalize='true')
    return pd.DataFrame([[f'TP = {tp} ({tpr:1.2%})', f'FN = {fn} ({fnr:1.2%})'], 
                         [f'FP = {fp} ({fpr:1.2%})', f'TN = {tn} ({tnr:1.2%})']],
                        index=['True', 'False'], 
                        columns=['Pred 1', 'Pred 0'])

print("number of errors on test dataset: " + str(sum(pred != y_test)))

conf_matrix(y_test,pred)

print(classification_report(y_test, pred)) 

number of errors on test dataset: 257


,Pred 1,Pred 0
True,TP = 118 (31.64%),FN = 255 (68.36%)
False,FP = 2 (0.05%),TN = 4010 (99.95%)


              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97      4012
         1.0       0.98      0.32      0.48       373

    accuracy                           0.94      4385
   macro avg       0.96      0.66      0.72      4385
weighted avg       0.94      0.94      0.93      4385



### Mitigations (Nulls, Duplicates)

In [11]:
# Mitigation
# random_sample.iloc[:,random_sample_target]
def split_label(dataset):
    x = dataset.drop(['is_promoted'], axis=1)
    y = dataset['is_promoted']
    return x, y

def split_label_index(dataset):
    x = dataset.drop([0], axis=1)
    y = dataset[0]
    return x, y

data_split2 =  DataSplit(random_sample, random_sample.columns.get_loc('is_promoted'), 0.9, seed, True, True, True, True)
train_data2, test_data2 = data_split2.Split()


dt_train2 =  DataTransformer(train_data2, train_data2.columns.get_loc('is_promoted'),DataTransformer.TransformerType.StandardScaler, None, seed)
train_data_t2 = dt_train2.Transform()


dt_test2 =  DataTransformer(test_data2, test_data2.columns.get_loc('is_promoted'),DataTransformer.TransformerType.StandardScaler, None, seed)
test_data_t2 = dt_test2.Transform()


x_train2, y_train2 = split_label_index(train_data_t2)
x_test2, y_test2 = split_label_index(test_data_t2)


clf2 = LGBMClassifier(n_estimators=50)
model2 = clf2.fit(x_train2, y_train2)
pred2 = model2.predict(x_test2)

def conf_matrix(y,pred):
    ((tn, fp), (fn, tp)) = metrics.confusion_matrix(y, pred)
    ((tnr,fpr),(fnr,tpr))= metrics.confusion_matrix(y, pred, normalize='true')
    return pd.DataFrame([[f'TP = {tp} ({tpr:1.2%})', f'FN = {fn} ({fnr:1.2%})'], 
                         [f'FP = {fp} ({fpr:1.2%})', f'TN = {tn} ({tnr:1.2%})']],
                        index=['True', 'False'], 
                        columns=['Pred 1', 'Pred 0'])

print("number of errors on test dataset: " + str(sum(pred2 != y_test2)))
conf_matrix(y_test2,pred2)
print(classification_report(y_test2, pred2)) 

number of errors on test dataset: 253


,Pred 1,Pred 0
True,TP = 117 (32.14%),FN = 247 (67.86%)
False,FP = 6 (0.16%),TN = 3824 (99.84%)


              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97      3830
         1.0       0.95      0.32      0.48       364

    accuracy                           0.94      4194
   macro avg       0.95      0.66      0.72      4194
weighted avg       0.94      0.94      0.93      4194



### Compare Results

In [12]:
# compare results before and after removing nulls

from sklearn.metrics import roc_auc_score

((tn2, fp2), (fn2, tp2)) = metrics.confusion_matrix(y_test2, pred2)
precision2 = round(tp2 /(tp2 + fp2), 5)
recall2 = round(tp2 /(tp2 + fn2), 5)

((tn, fp), (fn, tp)) = metrics.confusion_matrix(y_test, pred)
precision = round(tp /(tp + fp),5)
recall = round(tp /(tp + fn), 5)

preda = model.predict_proba(x_test)[:, 1]
roc_auc = round(roc_auc_score(y_test, preda),5)

preda2 = model2.predict_proba(x_test2)[:, 1]
roc_auc2 = round(roc_auc_score(y_test2, preda2),5)

               
def compare_results():
    return pd.DataFrame([[f'{precision}', f'{recall}', f'{roc_auc}'], 
                         [f'{precision2}', f'{recall2}', f'{roc_auc2}']],
                        columns=['Precision', 'Recall', 'roc_auc'], 
                        index=['No Mitigation', 'With Mitigation'])


compare_results()

,Precision,Recall,roc_auc
No Mitigation,0.98333,0.31635,0.91669
With Mitigation,0.95122,0.32143,0.90952


### Mitigation (Rebalance Dataset)

In [15]:
# Rebalance data

tomek = TomekLinks(sampling_strategy='auto')    
smote = SMOTE(sampling_strategy='auto', random_state= seed)    
smote_tomek = SMOTETomek(sampling_strategy='auto', random_state=seed)
# dataset - A Panda Data Frame representing the data to rebalance.   

# target – A string representing the name or the label column integer index (zero base) 
    # of the target feature to use as the classes for rebalancing the data.
    
# sampling_strategy  
    # 'minority': resample only the minority class. 
    # 'not minority': resample all classes but the minority class. 
    # 'not majority': resample all classes but the majority class. 
    # 'all': resample all classes. 
    # 'auto': equivalent to 'not majority'. 
    
# random_state - Control the randomization of the algorithm. 
    # ‘None’: the random number generator is the RandomState instance used by np.random.  
    # ‘If Int’: random_state is the seed used by the random number generator. 
    
# smote_tomek - The SMOTETomek object to use. If not given by Caller, a SMOTE object with default parameters will be given. 
    #  imblearn.combine.SMOTETomek 
    
# smote - The SMOTE object to use. If not given by Caller, a SMOTE object with default parameters will be given. 
    # imblearn_over_sampling.SMOTE
    
# tomek - The TomekLinks object to use. If not given by Caller, a TomekLinks object with sampling strategy=’all’ will be given.  imblearn.under_sampling.TomekLinks


train_data_rebalance3 =  DataRebalance(train_data2, train_data2.columns.get_loc('is_promoted'), 'auto', seed, None, smote)
train_data_r = train_data_rebalance3.Rebalance()

# test_data_rebalance3 =  DataRebalance(test_data2, test_data2.columns.get_loc('is_promoted'), 'auto', seed, None, smote)
# test_data_r = test_data_rebalance3.Rebalance()

x_train3, y_train3 = split_label(train_data_r)
x_test3, y_test3 = split_label(test_data2)

train_data2.shape
train_data_r.shape


(37737, 65)

(65180, 65)

### Accuracy Results

In [16]:

clf3 = LGBMClassifier(n_estimators=50)
model3 = clf3.fit(x_train3, y_train3)
pred3 = model3.predict(x_test3)

def conf_matrix(y,pred):
    ((tn, fp), (fn, tp)) = metrics.confusion_matrix(y, pred)
    ((tnr,fpr),(fnr,tpr))= metrics.confusion_matrix(y, pred, normalize='true')
    return pd.DataFrame([[f'TP = {tp} ({tpr:1.2%})', f'FN = {fn} ({fnr:1.2%})'], 
                         [f'FP = {fp} ({fpr:1.2%})', f'TN = {tn} ({tnr:1.2%})']],
                        index=['True', 'False'], 
                        columns=['Pred 1', 'Pred 0'])

print("number of errors on test dataset: " + str(sum(pred3 != y_test3)))
conf_matrix(y_test3,pred3)
print(classification_report(y_test3, pred3)) 


number of errors on test dataset: 263


,Pred 1,Pred 0
True,TP = 118 (32.42%),FN = 246 (67.58%)
False,FP = 17 (0.44%),TN = 3813 (99.56%)


              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3830
           1       0.87      0.32      0.47       364

    accuracy                           0.94      4194
   macro avg       0.91      0.66      0.72      4194
weighted avg       0.93      0.94      0.92      4194



### Compare Results

In [18]:
# compare results before and after removing nulls

from sklearn.metrics import roc_auc_score

((tn, fp), (fn, tp)) = metrics.confusion_matrix(y_test2, pred2)
precision2 = round(tp /(tp + fp), 5)
recall2 = round(tp2 /(tp2 + fn2), 5)


((tn3, fp3), (fn3, tp3)) = metrics.confusion_matrix(y_test3, pred3)
precision3 = round(tp3 /(tp3 + fp3),5)
recall3 = round(tp3 /(tp3 + fn3), 5)


preda3 = model3.predict_proba(x_test3)[:, 1]
roc_auc3 = round(roc_auc_score(y_test3, preda3),5)

preda2 = model2.predict_proba(x_test2)[:, 1]
roc_auc2 = round(roc_auc_score(y_test2, preda2),5)

               
def compare_results():
    return pd.DataFrame([[f'{precision}', f'{recall}', f'{roc_auc}'], 
                         [f'{precision2}', f'{recall2}', f'{roc_auc2}'],
                         [f'{precision3}', f'{recall3}', f'{roc_auc3}']],
                        columns=['Precision', 'Recall', 'roc_auc'], 
                        index=['No Mitigation', 'With Mitigation', 'With Mitigation & Rebalance'])


compare_results()

,Precision,Recall,roc_auc
No Mitigation,0.98333,0.31635,0.91669
With Mitigation,0.95122,0.32143,0.90952
With Mitigation & Rebalance,0.87407,0.32418,0.89784
